In [36]:
import os
import tempfile

from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv
import pinecone # noqa
import tiktoken
from PyPDF2 import PdfReader


In [3]:
load_dotenv()

True

In [82]:
# Convertir a texto los pdfs
pages = []
temp_dir = tempfile.TemporaryDirectory()
folder = 'data'

for file in os.listdir(folder):
    path_file = os.path.join(folder, file)
    
    reader = PdfReader(path_file)
    
    texto = "--------------------------\n"
    texto += "INICIO DEL DOCUMENTO\n"
    texto += "TITULO: " + file.replace("_", " ").replace(".pdf", " ") + '\n'
    
    for pagina in reader.pages:
        texto += pagina.extract_text() + '\n'
        
    texto += "FIN DEL DOCUMENTO\n"
    texto += "--------------------------\n"
    
    with open(f'text/{file.replace(".pdf", ".txt")}', 'w', encoding='utf-8') as archivo_txt:
        archivo_txt.write(texto)
        
# Crear loaders
folder = 'text'

for file in os.listdir(folder):
    print(file)
    
    file_path = os.path.join(folder, file)
    loader = TextLoader(file_path, encoding='utf-8')
    pages.extend(loader.load())

CONSTITUCIÓN_POLÍTICA_DE_LA_REPÚBLICA.txt
porpuesta_nueva_constitucion_2022.txt
porpuesta_nueva_constitucion_2023.txt


In [107]:
pages = []
temp_dir = tempfile.TemporaryDirectory()
folder = 'data'

for file in os.listdir(folder):
    print(file)
    
    file_path = os.path.join(folder, file)
    loader = PyPDFLoader(file_path=file_path)

    pages.extend(loader.load_and_split())

CONSTITUCIÓN_POLÍTICA_DE_LA_REPÚBLICA.pdf
porpuesta_nueva_constitucion_2023.pdf


In [112]:
len(pages)

376

In [104]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, 
                                               chunk_overlap=200,
                                               separators=["\n\n", "\n", " ", ""])
splits = text_splitter.split_documents(pages)

In [105]:
len(splits)

612

In [96]:
def consultar_costo_embedding(p):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in p])
    print(f"Total tokens {total_tokens}")
    print(f"Costo embedding {total_tokens / 1000 * 0.0001:.6f}")

In [97]:
consultar_costo_embedding(pages)

Total tokens 232306
Costo embedding 0.023231


In [98]:
api_key = os.getenv('OPENAI_API_KEY')
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [99]:
api_pinecone = os.getenv('PINECONE_API_KEY')
env_pinecone = os.getenv('PINECONE_ENV')

pinecone.init(
    api_key=api_pinecone,
    environment=env_pinecone
)
index_name = "constitution-idx"


In [100]:
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)

# docsearch = Pinecone.from_text([t.page_content for t in pages], embeddings, index_name=index_name)
docsearch = Pinecone.from_documents(pages, embeddings, index_name=index_name)
# docsearch = Pinecone.from_documents(splits, embeddings, index_name=index_name)

# docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [101]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 375}},
 'total_vector_count': 375}

In [102]:
docsearch.similarity_search(
    "educación",
    k=4
)

[Document(page_content=' \n 16\n La educación básica y la educación media son obligatorias, debiendo el Estado \nfinanciar un sistema gratuito con tal objeto, destinado a asegurar el acceso a ellas de \ntoda la población. En el caso de la educac ión media este sistema, en conformidad a la \nley, se extenderá hasta cumplir los 21 años de edad.49 \nCorresponderá al Estado, asimismo, fomentar el desarrollo de la educación en \ntodos sus niveles; estimular la investigación científica y tecnol ógica, la creación artística y \nla protección e incremento del patrimonio cultural de la Nación. \nEs deber de la comunidad contribuir al desarrollo y perfeccionamiento de la \neducación;  \n11°. La libertad de enseñanza incluye el derecho de abrir, organizar y mantener \nestablecimientos educacionales.  \nLa libertad de enseñanza no tiene otras limitaciones que las impuestas por la \nmoral, las buenas costumbres, el orden público y la seguridad nacional.  \nLa enseñanza reconocida oficialmente no po

In [ ]:
# retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 4})